# IMPORTING LIBRARIES

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm as cmaps
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from dateutil.relativedelta import relativedelta
from collections import Counter, OrderedDict
import itertools
import re

print("Packages loaded!")

Packages loaded!


# Pipeline Controller

hk_df=load_original_data() 
hk_df=lower_case_column_names(hk_df)
##hk_df=rename_columns(hk_df)
#hk_df=drop_columns(hk_df)

hk_df=load_original_data() 

hk_df= (hk_df.pipe(lower_case_column_names)\
.pipe(rename_columns)
 #.pipe(drop_columns)

)

# READ DATA INTO DATAFRAME FROM EXISTING .CSV

In [2]:
def load_original_data():
    """
    Read production data and parse into pandas dataframe.
    """
    latest_file = r"C:\Users\p.kollhof\Documents\IRONHACK_GitHub\DA_Midterm_Project\Data\Production_Data.csv"
    df_prod_whole = pd.read_csv(latest_file, sep=";", encoding = "ISO-8859-1")
    
    return df_prod_whole

df_prod_whole = load_original_data()

# DF CLEANING

## Delete unncessary/empty colums

In [3]:
def delete_unncessary_columns(df):
    """
    Delete columns only containing NaN values or columns with effectively useless data
    """
    #try:
    cols_to_drop = []
    if any(df_prod_whole.columns.str.contains("WT1")):
        cols_to_drop_1 = [col for col in df_prod_whole.columns if "WT1" in col]
        cols_to_drop.extend(cols_to_drop_1)
    if any(df_prod_whole.columns.str.contains("WT2")):
        cols_to_drop_2 = [col for col in df_prod_whole.columns if "WT2" in col]
        cols_to_drop.extend(cols_to_drop_2)
    if any(df_prod_whole.columns.str.contains("ET2")):
        cols_to_drop_3 = [col for col in df_prod_whole.columns if "ET2" in col]
        cols_to_drop.extend(cols_to_drop_3)
    if any(df_prod_whole.columns.str.contains("FT2")):
        cols_to_drop_4 = [col for col in df_prod_whole.columns if "FT2" in col]
        cols_to_drop.extend(cols_to_drop_4)
    if any(df_prod_whole.columns.str.contains("Schneiden_Ort")):
        cols_to_drop_5 = [col for col in df_prod_whole.columns if "Schneiden_Ort" in col]
        cols_to_drop.extend(cols_to_drop_5)
    if any(df_prod_whole.columns.str.contains("Schneiden_Geraet")):
        cols_to_drop_6 = [col for col in df_prod_whole.columns if "Schneiden_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_6)
    if any(df_prod_whole.columns.str.contains("Schleifen_Ort")):
        cols_to_drop_7 = [col for col in df_prod_whole.columns if "Schleifen_Ort" in col]
        cols_to_drop.extend(cols_to_drop_7)
    if any(df_prod_whole.columns.str.contains("Schleifen_Geraet")):
        cols_to_drop_8 = [col for col in df_prod_whole.columns if "Schleifen_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_8)
    if any(df_prod_whole.columns.str.contains("NS2_Ort")):
        cols_to_drop_9 = [col for col in df_prod_whole.columns if "NS2_Ort" in col]
        cols_to_drop.extend(cols_to_drop_9)
    if any(df_prod_whole.columns.str.contains("OeO_Ort")):
        cols_to_drop_10 = [col for col in df_prod_whole.columns if "OeO_Ort" in col]
        cols_to_drop.extend(cols_to_drop_10)
    if any(df_prod_whole.columns.str.contains("OeO_Geraet")):
        cols_to_drop_11 = [col for col in df_prod_whole.columns if "OeO_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_11)
        
    else:
        raise ValueError("Columns have already been deleted!")

    cols_to_drop.extend(["Versand_Intern", "SSMA_TimeStamp", "Assembley_Teflonschlauch-Charge"])
    df.drop(cols_to_drop, axis=1, inplace=True)

    #except:
        #raise ValueError("Deleting columns was unsuccessful!")
    
    return df

df_cleaned = delete_unncessary_columns(df_prod_whole)

## Make columns lowercase and fix syntax

In [4]:
def make_columns_lowercase(df):
    
    lower_cols = [i.lower().replace(" ", "_").replace("-", "_") for i in df.columns]
    df_cleaned.columns = lower_cols    
    
    return df_cleaned

df_cleaned = make_columns_lowercase(df_cleaned)

## Remove rows with XDC-ID < `68.000`

In [5]:
def delete_rows_with_XDC_ID_pre_68k(df):
    """
    Delete all entries before XDC-ID 68.000 due to them having wrong defect IDs.
    Only applies if dataframe actually has older entries with XDC-IDs < 68.000
    """
    
    df = df[df["pdc_nummer"]>=68000]
    
    return df

#df_cleaned = delete_rows_with_XDC_ID_pre_68k(df_cleaned)

## Remove rows with non-legit XDC-ID

In [6]:
def delete_non_legit_XDC_IDs(df):
    """
    Only keep entries with legit XDC-ID.
    Anything before 44145 is not usable.
    """
    
    df = df[df["pdc_nummer"]>=44145]
    
    return df

df_cleaned = delete_non_legit_XDC_IDs(df_cleaned)

## Cast `string-dates` into `datetime-dates`

In [7]:
def date_columns_transform_into_datetime(df):
    """
    Transform all dates of string type 'YYYY-MM-DD' into dt.datetime format.
    """
    date_cols = [col for col in df.columns if "datum" in col]

    for col in date_cols:
        if df[col].dtype == "object":
            df[col] = [dt.datetime.strptime(d,'%Y-%m-%d').date() if type(d)==str 
                       else np.nan if type(d)==float
                       else d
                       for d in df[col]]
            
    return df, date_cols

df_cleaned, date_columns = date_columns_transform_into_datetime(df_cleaned)

## Remove rows without entries in selected columns

In [8]:
def delete_rows_with_nan_in_selected_cols(df, cols):
    """
    Delete rows that do not have an entry in the given columns
    """
    for col in cols:
        df[col].dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    return df
    
df_cleaned = delete_rows_with_nan_in_selected_cols(df_cleaned, ["schmelzen_datum", "ns1_datum"])

## Substitute Product-`Familie` int-values with strings *("PDC", "PDC-C", "XDC")*

In [9]:
def substitute_familie_int_str(df):
    """
    Parse int values (1,2,3) into string values ("pdc", "pdc-c", "xdc")
    """
    
    if df["familie"].dtype == "int64":
        df["familie"] = ["pdc" if fam==1
                         else "pdc-c" if fam==2
                         else "xdc" 
                         for fam in df_cleaned["familie"]]
        
    return df

df_cleaned = substitute_familie_int_str(df_cleaned)

## Create `_Months` columns

In [10]:
def add_date_month_string_columns(df, date_columns):
    """
    Convert YYYY-MM-DD dt.datetime into new 'YYYY-MM' string month column. 
    """
    
    unwanted_dates = ['ts_in_datum', 'ts_out_datum', 'rt_bqc_in_datum', 'rt_bqc_out_datum']
    date_columns_wanted = [col for col in date_columns if col not in unwanted_dates]
    date_columns_months = [col.split("_")[0] + "_month" for col in date_columns_wanted]
    #date_columns_months = [col  for col in date_columns_months]
    
    #if "schmelzen_Mmnths" in df.columns:
    for idx, datum in enumerate(date_columns_wanted):
        df[date_columns_months[idx]] = [str(d)[:-3] for d in df[datum]]
    #else:
        #raise ValueError("Months columns have already been added!")
    return df

df_cleaned = add_date_month_string_columns(df_cleaned, date_columns)

## Turn string-comma-values into dot-float-values

In [11]:
def string_comma_to_dot_float(df):
    """
    Cast string-comma-values as dot-float-values
    """
    cols_to_use = ['ns1_winkel_vorne','ns1_winkel_hinten','ns1_durchmesser_vorne','ns1_durchmesser_hinten',
               "ns2_center_variation", "ns2_front_angle","ns2_curve","ns2_15xlength",
               "ns2_parabolic_factor","oeo_oeo", "piezo_phase"] 

    for column in cols_to_use:
        df[column] = [float(str(x).replace(',','.')) for x in df[column]] 
     
    return df

df_cleaned = string_comma_to_dot_float(df_cleaned)

## Clean personal (operator) initials

In [57]:
def clean_personal_names(df):
    """
    Func descr
    """
    
    replacements = {'GD': 'GDH',
                    'MFW': 'MFE',
                    "ME": "MFE",
                    "MF": "MFE",
                    "LAE": "AL",
                    "SF": "SFZ",
                    "DC": "AC",
                    "THANH": "TT",
                    "?": np.nan}
    
    personal_cols = [col for col in df_cleaned.columns if "personal" in col]
    
    for col in personal_cols:
        op_list = []

        for op in df_cleaned["ft1_personal"]:
            if type(op)==str and op.isalpha():
                op_list.append(op.upper())
            elif op=="?":
                op_list.append(np.nan)
            elif type(op)==str and "." in op:
                op_list.append(np.nan)
            elif type(op)==str and not op.isalpha() and op!="?":
                op_list.append(op[:re.search(r'\W+', s).start()].upper())
            else:
                op_list.append(np.nan)

        op_list = [replacements.get(word, word) for word in op_list]

        df[col] = op_list
        
    return df
    
df_cleaned = clean_personal_names(df_cleaned)